In [14]:
import sys
sys.path.append('/home/cdsw/utils')

In [1]:
import language_tool_python

# Initialize LanguageTool with the specific language model (e.g., 'en-US')
tool = language_tool_python.LanguageTool('en-US')

# Disable specific rules by their IDs
tool.disabled_rules = ['MORFOLOGIK_RULE_EN_US']  # Disable spell-checking

# Check text
text = "Skystriker and Marko Ramius are part of the story."
matches = tool.check(text)

# Output the results
for match in matches:
    print(f"Rule ID: {match.ruleId}, Message: {match.message}")


In [3]:
import language_tool_python
from deterministic_preprocessor import *
checker = DeterministicPreprocessor()
# Initialize the LanguageTool object
tool = language_tool_python.LanguageTool('en-AU')

# Disable the rule for spell-checking (e.g., "MORFOLOGIK_RULE_EN_US")
#checker.tool.disabled_rules = ['MORFOLOGIK_RULE_EN_AU']
tool.disabled_rules = ['MORFOLOGIK_RULE_EN_AU']
# Check text without this rule
text = "Rear Admiral Tom Kazansky AO, RAN"
print("Deterministic")
matches = checker.tool.check(text)
for match in matches :
    print(f"Rule ID : {match.ruleId}, Message: {match.message}")
# Output the results
for match in matches:
    print(match)

print("normal")

matches = tool.check(text)

# Output the results
for match in matches:
    print(match)

Deterministic
normal


In [21]:
# Assume doc_object is an existing Document object
from docx import Document
from  doc_formatter import DocumentFormatter
from copy import deepcopy
#formatter = DocumentFormatter(mydocx_doc, 'Vish Rajagopalan')

# Add a comment to the first paragraph
# for para in formatter.document.paragraphs :
# #formatter.add_comment(paragraph, "This is a comment.")
#     formatter.add_comment( paragraph, "This is a para comment.")

# para_0 = formatter.document.paragraphs[0]
# formatter.add_comment( para_0, "This is a para comment.")

# para_2 = formatter.document.paragraphs[2]
# formatter.add_comment( para_2, "This is a para comment.")

# para_6 = formatter.document.paragraphs[6]
# formatter.add_comment( para_6, "This is a para comment.")
def function1_addcomments(doc):
    formatter = DocumentFormatter(doc, "Vish")
    for  _, para in enumerate(formatter.document.paragraphs, start =0):
        if len(para.text) != 0:
            formatter.add_comment(para, "This is a para comment")

    for table in formatter.document.tables:
        print("In TABLE")
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    # Add an asterisk (*) to the end of each cell paragraph
                    print(paragraph.text)
                    paragraph.add_run('*P*T*')
                    if len(paragraph.text) != 0:
                        formatter.add_comment(paragraph, "This is a table comment.")
    return doc
 
def function2_addcomments(doc):   
    formatter = DocumentFormatter(doc,"Vish") 
    for  _, para in enumerate(formatter.document.paragraphs, start =0):
        if len(para.text) != 0:
            formatter.add_comment(para, "This is a para comment 2")


    for table in formatter.document.tables:
        print("In TABLE")
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    # Add an asterisk (*) to the end of each cell paragraph
                    print(paragraph.text)
                    paragraph.add_run('*P*T*')
                    if len(paragraph.text) != 0:
                        formatter.add_comment(paragraph, "This is a table comment 2")     
    return doc        
path = '/home/cdsw/data/simple-word-file-with-table.docx'
doc =  Document(path)
modified_doc = deepcopy(doc)
modified_doc = function1_addcomments(modified_doc)
modified_doc = function2_addcomments(modified_doc)
# Save the modified document
modified_doc.save('/home/cdsw/data/comment_doc.docx')


comment_id:1
Added Comment to Document :  This is a para comment
comment_id:2
Added Comment to Document :  This is a para comment
comment_id:3
Added Comment to Document :  This is a para comment
comment_id:4
Added Comment to Document :  This is a para comment
comment_id:5
Added Comment to Document :  This is a para comment
comment_id:6
Added Comment to Document :  This is a para comment
comment_id:7
Added Comment to Document :  This is a para comment
comment_id:8
Added Comment to Document :  This is a para comment
comment_id:9
Added Comment to Document :  This is a para comment
In TABLE
Spelling Errors
comment_id:10
Added Comment to Document :  This is a table comment.
Grammar Errors
comment_id:11
Added Comment to Document :  This is a table comment.
This is a mistke in spelling
comment_id:12
Added Comment to Document :  This is a table comment.
This is examples of grammar mistakes
comment_id:13
Added Comment to Document :  This is a table comment.
The wether was beautifull on our vaca

In [16]:
for index  ,para in enumerate(formatter.document.paragraphs, start=0):
    print(index, para.text, len(para.text))

0 HEADING 7
1  0
2 SECTION 1  10
3 I am trying something bold here.  33
4  0
5  0
6 SECTION 2  10
7  0
8 Product Details 15
9  0
10 I saw a smaple product. These products is below expected quality. \n I start to understand what he said is quite     right.  The correct value is 12.5  % and 3   r d position. The time is 10:30 am  197
11                    The length of the object is 845mm and cost $1m. The sunrise happens at 0750 EST and visibily is 35 km.      The right way to summarize and favor some advisors is still to be found 198
12  0
13  0
14 Table Section 13
15 Some table to be addd latr here by I. I am trying t  o  make a long  setence to see if the LLM is able to capture some of these                       errors such as 2    nd  or 4    th or 6.5   %  196
16  0
17  0
18  0


In [12]:


def load_mappings(file_path):
    """Load abbreviations or expansions from a file."""
    mapping_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            if '||' in line:
                key, value = line.strip().split('||')
                mapping_dict[key] = value
    return mapping_dict

def process_abbreviations(text, abbreviations_dict, corrections):
    """Replace abbreviations with their expanded forms."""
    modified_text = text

    for abbr, expansion in abbreviations_dict.items():
        if abbr in modified_text:
            # Replace abbreviation with the expanded form
            modified_text = modified_text.replace(abbr, expansion)

            # Add correction details
            correction = {
                "id": len(corrections) + 1,
                "type": "abbreviation",
                "original_text": abbr,
                "modified_text": expansion
            }
            corrections.append(correction)

    return modified_text, corrections

def process_expansions(text, expansions_dict, corrections):
    """Replace full names with their abbreviations."""
    modified_text = text

    for full_name, acronym in expansions_dict.items():
        if full_name in modified_text:
            # Replace full name with its acronym
            modified_text = modified_text.replace(full_name, acronym)

            # Add correction details
            correction = {
                "id": len(corrections) + 1,
                "type": "expansion",
                "original_text": full_name,
                "modified_text": acronym
            }
            corrections.append(correction)

    return modified_text, corrections

# Loading abbreviation and expansion mappings from files
abbreviations_dict = load_mappings('abbreviations.txt')
expansions_dict = load_mappings('expansions.txt')

# Example usage
original_text = "CAPT Jack Sparrow and CMDR John Rambo was a great guy in Ministry of Defence Industries."

corrections = []
# Process abbreviations
modified_text_abbr, corrections = process_abbreviations(original_text, abbreviations_dict, corrections)

# Process expansions
final_modified_text, corrections = process_expansions(modified_text_abbr, expansions_dict, corrections)

# Final result
result = {
    "original_text": original_text,
    "modified_text": final_modified_text,
    "corrections": corrections
}

print(result)




Original Text: CAPT John Sparrow was a great guy in Ministry of Defence Industries.
Modified Text: Captain John Sparrow was a great guy in MINDI.
Corrections: [{'id': 1, 'mode': 'abbreviation', 'original_text': 'CAPT', 'modified_text': 'Captain'}, {'id': 1, 'mode': 'expansion', 'original_text': 'Ministry of Defence Industries', 'modified_text': 'MINDI'}]


In [12]:
import os
class DeterministicProcessor:
    def __init__(self, abbreviations_file, expansions_file):
        """
        Initializes the processor with abbreviation and expansion mappings from the given files.
        """
        self.abbreviations_dict = self.load_mappings(abbreviations_file)
        self.expansions_dict = self.load_mappings(expansions_file)
        self.corrections = []


    def load_mappings(self,file_path):
        mapping_dict = {}
        try:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")

            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if not line or line.startswith('#'):  # Skip empty lines and comments
                        continue
                    try:
                        key, value = line.split('||')
                        mapping_dict[key] = value
                    except ValueError as e:
                        raise ValueError(f"Invalid line format in {file_path}: {line}") from e
        except Exception as e:
            raise Exception(f"Error loading mapping dictionary from {file_path}: {e}") from e
        return mapping_dict

    def expand_abbreviations(self, text):
        """
        Replace abbreviations with their expanded forms.
        Returns modified text and a list of corrections made.
        """

        modified_text = text

        for abbr, expansion in self.abbreviations_dict.items():
            if abbr in modified_text:
                modified_text = modified_text.replace(abbr, expansion)
                correction = {
                    "id": len(self.corrections) + 1,
                    "mode": "abbreviation",
                    "original_text": abbr,
                    "modified_text": expansion
                }
                self.corrections.append(correction)

        return modified_text, tuple(self.corrections)

    def reduce_expansions(self, text):
        """
        Replace full names with their abbreviations.
        Returns modified text and a list of corrections made.
        """
        modified_text = text

        for full_name, acronym in self.expansions_dict.items():
            if full_name in modified_text:
                modified_text = modified_text.replace(full_name, acronym)
                correction = {
                    "id": len(self.corrections) + 1,
                    "mode": "expansion",
                    "original_text": full_name,
                    "modified_text": acronym
                }
                self.corrections.append(correction)

        return modified_text,tuple(self.corrections)

    def process_text(self, text):
        """
        Process the text by expanding abbreviations and reducing expansions.
        Returns the modified text and a combined list of all corrections.
        """
        # Step 1: Expand abbreviations
        modified_text_abbr, _ = self.expand_abbreviations(text)

        # Step 2: Reduce full names to acronyms
        final_modified_text, _ = self.reduce_expansions(modified_text_abbr)


        # Return the final modified text and all corrections
        return final_modified_text

# Example usage
# Assuming 'abbreviations.txt' and 'expansions.txt' files are available

uploads_path = os.path.abspath("/home/cdsw/uploads")
processor = DeterministicProcessor(os.path.join(uploads_path, 'abbreviations.txt'), os.path.join(uploads_path, 'expansions.txt'))

original_text = "CAPT John Sparrow and CMDR Vish Rajagopalan was a great guy in Ministry of Defence Industries."

# Process the text
modified_text = processor.process_text(original_text)

# Display results
print("Original Text:", original_text)
print("Modified Text:", modified_text)
print("Corrections:", processor.corrections)





Original Text: CAPT John Sparrow and CMDR Vish Rajagopalan was a great guy in Ministry of Defence Industries.
Modified Text: Captain John Sparrow and Commander Vish Rajagopalan was a great guy in MINDI.
Corrections: [{'id': 1, 'mode': 'abbreviation', 'original_text': 'CAPT', 'modified_text': 'Captain'}, {'id': 2, 'mode': 'abbreviation', 'original_text': 'CMDR', 'modified_text': 'Commander'}, {'id': 3, 'mode': 'expansion', 'original_text': 'Ministry of Defence Industries', 'modified_text': 'MINDI'}]
